In [1]:
def composites(wave_ds,precip_ds):    
    wavedict = {}
    
    time_phase = wave_ds["time"].astype("datetime64[D]")
    time_precip = precip_ds["time"].astype("datetime64[D]")

    phases = wave_ds.groupby("phase").groups

    for phase in phases:
        phase_indices = np.where(phase == wave_ds["phase"])[0]
        precip_values = [
            precip_ds.where(time_phase[i] == time_precip).dropna(dim = "time").values[0]
            for i in phase_indices
        ]
        precip_time = [
            precip_ds["time"].where(time_phase[i] == time_precip).dropna(dim = "time").values[0]
            for i in phase_indices
        ]
        wavedict[str(int(phase))] = [precip_time, precip_values]
#         wavedict["time-"+str(int(phase))] = precip_time
    return(wavedict)

def is_seas(month, seas):
    if seas=='jja':
        return (month >= 6) & (month <= 8)
    elif seas=='djf':
        return (month >= 12) | (month <= 2)
    elif seas=='son':
        return (month >= 9) & (month <= 11)
    elif seas=='mam':
        return (month >= 3) & (month <= 5)
        
def placehold(array,fillval):
    if len(array)==0:
        return(fillval)
    else: return(array)

In [13]:
import xarray as xr
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

seas = 'djf'
if seas=='mam':
    seasname = "SHAutumn"
elif seas=='djf':
    seasname = "SHSummer"

In [14]:
diri = "/Users/fadhlilrizki/Documents/PhD/thesis/data/"
diro = "/Users/fadhlilrizki/Documents/PhD/thesis/dataout/"

In [15]:
ds   = xr.open_dataset(diri+"agcd_v1_precip_calib_r025_daily_1981-2020.masked.nc")
ds

<xarray.Dataset>
Dimensions:    (time: 14396, bnds: 2, lon: 177, lat: 138)
Coordinates:
  * time       (time) datetime64[ns] 1981-01-01T09:00:00 ... 2020-05-31T09:00:00
  * lon        (lon) float32 112.0 112.2 112.5 112.8 ... 155.2 155.5 155.8 156.0
  * lat        (lat) float32 -44.5 -44.25 -44.0 -43.75 ... -10.75 -10.5 -10.25
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    precip     (time, lat, lon) float32 ...
Attributes: (12/35)
    CDI:                       Climate Data Interface version 2.1.0 (https://...
    source:                    Data is from site-based observation data recor...
    institution:               Australian Bureau of Meteorology, GPO Box 1289...
    Conventions:               CF-1.6, ACDD-1.3
    geospatial_lat_min:        -44.525
    geospatial_lat_max:        -9.975
    ...                        ...
    description:               This AGCD data is a snapshot of the operationa...
    date_issued:               2020-12-02 05:42:26
    attribution:               Data should be cited as : Australian Bureau of...
    copyright:                 (C) Copyright Commonwealth of Australia 2020, ...
    history:                   Mon Jul 10 21:07:57 2023: cdo mul /Users/fadhl...
    CDO:                       Climate Data Operators version 2.1.0 (https://...

In [25]:
startLat = -16.75
startLon = 145.0
endLat = -20.0
endLon = 147.75
n = 100
x = xr.DataArray(np.linspace(startLon,endLon,n), dims='along_course')
y = xr.DataArray(np.linspace(startLat,endLat,n), dims='along_course')
precip_profile = ds['precip'].interp(lon=x, lat=y, method='nearest')

/Users/fadhlilrizki/opt/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/Users/fadhlilrizki/opt/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/Users/fadhlilrizki/opt/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/Users/fadhlilrizki/opt/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing metho

In [26]:
precip_profile

<xarray.DataArray 'precip' (time: 14396, along_course: 100)>
array([[1.2773422e+01, 1.2773422e+01, 1.2773422e+01, ..., 6.8944341e-01,
        6.8944341e-01, 6.8944341e-01],
       [1.3711417e+01, 1.3711417e+01, 1.3711417e+01, ..., 8.7101498e+00,
        8.7101498e+00, 8.7101498e+00],
       [6.1568747e+00, 6.1568747e+00, 6.1568747e+00, ..., 5.2422233e+00,
        5.2422233e+00, 5.2422233e+00],
       ...,
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [3.9082988e-05, 3.9082988e-05, 3.9082988e-05, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [8.3994837e-03, 8.3994837e-03, 8.3994837e-03, ..., 2.5040954e-02,
        2.5040954e-02, 2.5040954e-02]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01T09:00:00 ... 2020-05-31T09:00:00
    lon      (along_course) float64 145.0 145.0 145.1 ... 147.7 147.7 147.8
    lat      (along_course) float64 -16.75 -16.78 -16.82 ... -19.93 -19.97 -20.0
Dimensions without coordinates: along_course
Attributes:
    standard_name:                 lwe_thickness_of_precipitation_amount
    long_name:                     Daily precipitation
    units:                         mm
    cell_methods:                  time: sum
    analysis_version_number:       3.01
    number_of_stations_reporting:  6600
    frequency:                     daily
    length_scale_for_analysis:     80.0
    coverage_content_type:         physicalMeasurement